# Farnebäck Flow

In [1]:
# init prerequisites
%reload_ext autoreload
%autoreload 2
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os, re, math, time, random
from pprint import pprint

import util
from util import print_warn, horline

print("OpenCV {}".format(cv.__version__))
print("Numpy {}".format(np.__version__))

print_warn("Warning Text!")
horline()


OpenCV 3.4.1
Numpy 1.16.2
 Warning Text! 
________________________________________________________________________________________________________________________


In [4]:
# gasonics production equipment
config = dict(
    inputfolder=r".\input\classes_r3",
    outputfolder=r".\output\flow",  
)
pprint(config)

os.makedirs(config["outputfolder"], exist_ok=True)

{'inputfolder': '.\\input\\classes_r3', 'outputfolder': '.\\output\\flow'}


In [5]:
# collect input files
paths, labels, label_mapping = util.recursive_collect(config["inputfolder"])
input_images = []
prev = None

shuffle = list(zip(paths, labels))
# shuffle or sort
#random.shuffle(shuffle)
shuffle = sorted(shuffle, key = lambda x: x[0].split("\\")[-1])
paths, labels = zip(*shuffle[300:])

# one time initialization
orb = cv.ORB_create(nfeatures=1500, scaleFactor=2, patchSize=31, edgeThreshold=31)
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

# read input image and store for future use
robot_light = cv.imread("img_gasonics/robot_light.png", cv.IMREAD_GRAYSCALE)

# find keypoints
robot_light_kp = orb.detect(robot_light,None)
# compute the descriptors
robot_light_kp, robot_light_des = orb.compute(robot_light, robot_light_kp)
print("  found {} keypoints for search image".format(len(robot_light_kp)))

cv.destroyAllWindows()
out = None
for i, path, label in zip(range(len(paths)), paths, labels):
    print(i, path, label)
    start_time = time.clock()
    
    # read input image and store for future use
    img = cv.imread(path, cv.IMREAD_COLOR)
    img = cv.resize(img, (int(img.shape[1]*0.5), int(img.shape[0]*0.5)))
    img = cv.medianBlur(img, 13)
    cv.imshow('img', img)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    if prev is None:
        prev = gray
        hsv = np.zeros_like(img)
        hsv[:,:,1] = 255
    
#    print(prev.shape, gray.shape)
#    print(hsv.shape)
        
    flow = cv.calcOpticalFlowFarneback(prev,gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[:,:,0], flow[:,:,1])
    hsv[:,:,0] = ang*180/np.pi/2
    hsv[:,:,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('flow', bgr)
    
    if out is None:
        outh,outw,_ = bgr.shape
        out = cv.VideoWriter(os.path.join(config["outputfolder"], "farneback-orb.avi"), cv.VideoWriter_fourcc(*'MJPG'), 15, (outw,outh))
    vidimg = cv.addWeighted(bgr, 0.5, img, 0.5, 0)
    cv.imshow('save', vidimg)
    out.write(vidimg)
    prev = gray
    
    #img = cv.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    #img = cv.equalizeHist(img)
    #img = clahe.apply(img)
    #img = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 127, 2)
    #input_images.append(img)
    
    # find keypoints
    kp = orb.detect(img,None)
    
    # compute the descriptors
    kp, des = orb.compute(img, kp)
    #print("  found {} keypoints".format(len(kp)))
    
    #matches = bf.match(robot_light_des,des)
    #matches = sorted(matches, key = lambda x:x.distance)
    #img3 = cv.drawMatches(robot_light,robot_light_kp,img,kp,matches[:10],img,flags=2)
    #plt.imshow(img3),plt.show()
    
    # show keypoint location
    img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=4)
    end_time = time.clock()
    print("  computation for {} took {:.2f}ms".format(path, (end_time - start_time)*1000.0))
    
    #plt.subplot(1,3,3)
    #plt.imshow(img2), plt.show()
    cv.imshow('keypoints', img2)
    k = cv.waitKey(20) & 0xff
    if k == 27:
        break
if out is not None:
    out.release()
cv.destroyAllWindows()

  found 0 keypoints for search image
0 .\input\classes_r3\3.2_cassette_location_unknown\gray_89071.0.png 4
  computation for .\input\classes_r3\3.2_cassette_location_unknown\gray_89071.0.png took 138.19ms
1 .\input\classes_r3\3.2_cassette_location_unknown\gray_89072.0.png 4


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


  computation for .\input\classes_r3\3.2_cassette_location_unknown\gray_89072.0.png took 78.59ms
2 .\input\classes_r3\3.3_cassette_location_wafer\gray_89073.0.png 5
  computation for .\input\classes_r3\3.3_cassette_location_wafer\gray_89073.0.png took 78.15ms
3 .\input\classes_r3\3.3_cassette_location_wafer\gray_89074.0.png 5
  computation for .\input\classes_r3\3.3_cassette_location_wafer\gray_89074.0.png took 76.47ms
4 .\input\classes_r3\3.3_cassette_location_wafer\gray_89075.0.png 5
  computation for .\input\classes_r3\3.3_cassette_location_wafer\gray_89075.0.png took 76.00ms
5 .\input\classes_r3\3.3_cassette_location_wafer\gray_89076.0.png 5
  computation for .\input\classes_r3\3.3_cassette_location_wafer\gray_89076.0.png took 74.61ms
6 .\input\classes_r3\2.2_moving_wafer\gray_89077.0.png 2
  computation for .\input\classes_r3\2.2_moving_wafer\gray_89077.0.png took 75.24ms
7 .\input\classes_r3\2.2_moving_wafer\gray_89078.0.png 2
  computation for .\input\classes_r3\2.2_moving_wafer

  computation for .\input\classes_r3\4.1_processchamber_empty\gray_89276.0.png took 79.29ms
55 .\input\classes_r3\4.1_processchamber_empty\gray_89277.0.png 6
  computation for .\input\classes_r3\4.1_processchamber_empty\gray_89277.0.png took 79.07ms
56 .\input\classes_r3\4.1_processchamber_empty\gray_89278.0.png 6
  computation for .\input\classes_r3\4.1_processchamber_empty\gray_89278.0.png took 78.96ms
57 .\input\classes_r3\4.1_processchamber_empty\gray_89279.0.png 6
  computation for .\input\classes_r3\4.1_processchamber_empty\gray_89279.0.png took 78.70ms
58 .\input\classes_r3\4.1_processchamber_empty\gray_89356.0.png 6
  computation for .\input\classes_r3\4.1_processchamber_empty\gray_89356.0.png took 78.26ms
59 .\input\classes_r3\4.1_processchamber_empty\gray_89357.0.png 6
  computation for .\input\classes_r3\4.1_processchamber_empty\gray_89357.0.png took 76.26ms
60 .\input\classes_r3\4.1_processchamber_empty\gray_89358.0.png 6
  computation for .\input\classes_r3\4.1_processcham

  computation for .\input\classes_r3\3.2_cassette_location_unknown\gray_89405.0.png took 77.22ms
108 .\input\classes_r3\3.2_cassette_location_unknown\gray_89406.0.png 4
  computation for .\input\classes_r3\3.2_cassette_location_unknown\gray_89406.0.png took 75.55ms
109 .\input\classes_r3\3.2_cassette_location_unknown\gray_89407.0.png 4
  computation for .\input\classes_r3\3.2_cassette_location_unknown\gray_89407.0.png took 75.60ms
110 .\input\classes_r3\3.1_cassette_location_empty\gray_89408.0.png 3
  computation for .\input\classes_r3\3.1_cassette_location_empty\gray_89408.0.png took 77.18ms
111 .\input\classes_r3\3.1_cassette_location_empty\gray_89409.0.png 3
  computation for .\input\classes_r3\3.1_cassette_location_empty\gray_89409.0.png took 75.45ms
112 .\input\classes_r3\3.1_cassette_location_empty\gray_89410.0.png 3
  computation for .\input\classes_r3\3.1_cassette_location_empty\gray_89410.0.png took 75.06ms
113 .\input\classes_r3\3.1_cassette_location_empty\gray_89411.0.png 3


In [ ]:
cv.destroyAllWindows()